In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility --> random 값을 매번 같은 것을 반환함
#sklearn 추가
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

In [2]:
#여기는 강의안 따라 만들어 본 것
tf.set_random_seed(777)  # for reproducibility --> random 값을 매번 같은 것을 반환함
xy_train = np.loadtxt('Heart_Train_Changed.csv',skiprows=1,delimiter=',',dtype=np.float32)
xy_test = np.loadtxt('Heart_Test_Changed.csv',skiprows=1,delimiter=',',dtype=np.float32)
#input data를 normalize 하지 않으면 학습이 제대로 안된다!

# 이런식으로 함수를 만들어도 되고 sklearn을 통해서 normalize 함수를 불러와도 되고
# def MinMaxScaler(data):
#     numerator = data - np.min(data, 0)
#     denominator = np.max(data, 0) - np.min(data, 0)
#     # noise term prevents the zero division
#     return numerator / (denominator + 1e-5)
# xy = MinMaxScaler(xy)
scaler = MinMaxScaler(feature_range=(0,1))
xy_train= scaler.fit_transform(xy_train)
xy_test = scaler.fit_transform(xy_test)


x_data_train = xy_train[:,0:-1]
y_data_train = xy_train[:,[-1]]
x_data_test = xy_test[:,0:-1]
y_data_test = xy_test[:,[-1]]

# x_data_train = xy[:250,0:-1]
# x_data_test = xy[250:,0:-1]
# y_data_train = xy[:250,[-1]]
# y_data_test = xy[250:,[-1]]


# print(x_data_train)
print(x_data_train.shape)
# print(y_data_train)
print(y_data_train.shape)

# print(x_data_test)
print(x_data_test.shape)
# print(y_data_test)
print(y_data_test.shape)

X = tf.placeholder(tf.float32,shape=[None,13])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([13,1]),name='weight')
b = tf.Variable(tf.random_normal([1]),name='bias')

hypothesis = tf.sigmoid(tf.matmul(X,W)+b)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

#predicted 값을 설정하기위해서 hypothesis 값을 어떤식으로 조정해야하지?
#어떻게 해야 좋은게
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], 
                               feed_dict={X: x_data_train, Y: y_data_train})
#         if step%100 == 0:
#             print(step,cost_val)
    
    h, c, a = sess.run([hypothesis,predicted, accuracy],
                               feed_dict = {X:x_data_train,Y:y_data_train})
    print("\n(test) hypothesis: ",h,"\nCorrect (Y):",c,"\nAccuracy ",a)
    
    h, c, a = sess.run([hypothesis,predicted, accuracy],
                               feed_dict = {X:x_data_test,Y:y_data_test})
    print("\n(training) hypothesis: ",h,"\nCorrect (Y):",c,"\nAccuracy ",a)


(235, 13)
(235, 1)
(57, 13)
(57, 1)

(test) hypothesis:  [[0.8650817 ]
 [0.02060679]
 [0.02473757]
 [0.72879183]
 [0.9183578 ]
 [0.86647666]
 [0.3377589 ]
 [0.71807706]
 [0.21057904]
 [0.07346612]
 [0.76286346]
 [0.83954906]
 [0.7596277 ]
 [0.8851917 ]
 [0.50462645]
 [0.7243991 ]
 [0.95276815]
 [0.84872854]
 [0.50332594]
 [0.9734896 ]
 [0.41468444]
 [0.02477533]
 [0.853901  ]
 [0.06429639]
 [0.90944946]
 [0.13517278]
 [0.890786  ]
 [0.7951417 ]
 [0.6105473 ]
 [0.8291048 ]
 [0.8971    ]
 [0.8940475 ]
 [0.2086123 ]
 [0.7084897 ]
 [0.9244883 ]
 [0.5568889 ]
 [0.5544909 ]
 [0.8416747 ]
 [0.12545979]
 [0.03256509]
 [0.567205  ]
 [0.5370765 ]
 [0.52032036]
 [0.56877387]
 [0.01448339]
 [0.978228  ]
 [0.20061085]
 [0.68400544]
 [0.9507536 ]
 [0.32605702]
 [0.0260075 ]
 [0.6086015 ]
 [0.6401905 ]
 [0.93022954]
 [0.04632321]
 [0.8666786 ]
 [0.7397519 ]
 [0.16946724]
 [0.77048135]
 [0.8796705 ]
 [0.25036722]
 [0.8638315 ]
 [0.8799261 ]
 [0.84247696]
 [0.87659466]
 [0.5560211 ]
 [0.06672698]
 [0.2

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils


# error fix link 
# :https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous 
lab_enc = preprocessing.LabelEncoder()

#np.ravel 이게 뭔지 잘 모르겠네

training_scores_y = lab_enc.fit_transform(np.ravel(y_data_train))
print(np.ravel(y_data_train).shape)
lr = LogisticRegression(solver='lbfgs')

lr.fit(x_data_train,training_scores_y)
print("Test Accuracy (training) {:.2f}%".format(lr.score(x_data_train,y_data_train)*100))
print("Test Accuracy (test) {:.2f}%".format(lr.score(x_data_test,y_data_test)*100))

(235,)
Test Accuracy (training) 86.38%
Test Accuracy (test) 80.70%


In [4]:
ridge = Ridge(alpha=0.01).fit(x_data_train,y_data_train)
print("Test Accuracy (training): {:.2f}".format(ridge.score(x_data_train, y_data_train)))
print("Test Accuracy (test): {:.2f}".format(ridge.score(x_data_test, y_data_test)))

Test Accuracy (training): 0.55
Test Accuracy (test): 0.49


In [11]:
#StandardScaler
xy_train = np.loadtxt('Heart_Train_Changed.csv',skiprows=1,delimiter=',',dtype=np.float32)
xy_test = np.loadtxt('Heart_Test_Changed.csv',skiprows=1,delimiter=',',dtype=np.float32)

scaler = StandardScaler()
xy_train= scaler.fit_transform(xy_train)
xy_test = scaler.fit_transform(xy_test)

x_data_train = xy_train[:,0:-1]
y_data_train = xy_train[:,[-1]]
x_data_test = xy_test[:,0:-1]
y_data_test = xy_test[:,[-1]]

print(x_data_train)

X = tf.placeholder(tf.float32,shape=[None,13])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([13,1]),name='weight')
b = tf.Variable(tf.random_normal([1]),name='bias')

hypothesis = tf.sigmoid(tf.matmul(X,W)+b)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

#predicted 값을 설정하기위해서 hypothesis 값을 어떤식으로 조정해야하지?
#어떻게 해야 좋은게
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], 
                               feed_dict={X: x_data_train, Y: y_data_train})
        if step%100 == 0:
            print(step,cost_val)
    
    h, c, a = sess.run([hypothesis,predicted, accuracy],
                               feed_dict = {X:x_data_train,Y:y_data_train})
    print("\n(test) hypothesis: ",h,"\nCorrect (Y):",c,"\nAccuracy ",a)
    
    h, c, a = sess.run([hypothesis,predicted, accuracy],
                               feed_dict = {X:x_data_test,Y:y_data_test})
    print("\n(training) hypothesis: ",h,"\nCorrect (Y):",c,"\nAccuracy ",a)


[[63.  1.  3. ...  0.  1.  1.]
 [67.  1.  0. ...  3.  2.  0.]
 [67.  1.  0. ...  2.  3.  0.]
 ...
 [44.  1.  0. ...  0.  1.  0.]
 [63.  1.  0. ...  2.  3.  0.]
 [63.  0.  0. ...  0.  2.  0.]]
[[ 0.931073    0.7116251   1.9825319  ... -0.7168576  -2.336036
   0.87573814]
 [ 1.3735445   0.7116251  -0.9232581  ...  2.5021527  -0.5708641
  -1.1418939 ]
 [ 1.3735445   0.7116251  -0.9232581  ...  1.4291493   1.1943078
  -1.1418939 ]
 ...
 [-1.1706665   0.7116251  -0.9232581  ... -0.7168576  -2.336036
  -1.1418939 ]
 [ 0.931073    0.7116251  -0.9232581  ...  1.4291493   1.1943078
  -1.1418939 ]
 [ 0.931073   -1.4052343  -0.9232581  ... -0.7168576  -0.5708641
  -1.1418939 ]]
[[ 0.931073    0.7116251   1.9825319  ...  2.25724    -0.7168576
  -2.336036  ]
 [ 1.3735445   0.7116251  -0.9232581  ...  0.6449257   2.5021527
  -0.5708641 ]
 [ 1.3735445   0.7116251  -0.9232581  ...  0.6449257   1.4291493
   1.1943078 ]
 ...
 [-1.1706665   0.7116251  -0.9232581  ...  2.25724    -0.7168576
  -2.336036  ]